In [0]:
import pandas as pd
import numpy as np
import datetime
from time import strptime
import string
import re
pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)

In [0]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [0]:
sharks = pd.read_excel('/kaggle/input/shark-attacks-gsaf5/GSAF5 (sharks).xls', sort =False)

In [0]:
sharks.head()

In [0]:
sharks.dropna(subset = ["Date"], inplace=True)

In [0]:
sharks.columns

In [0]:
sharks = sharks.drop(columns = [ 'Unnamed: 23', 'href formula', 'pdf', 'Case Number.1','Case Number.2', 'original order'])

## Clean 'Country'

In [0]:
#set(sharks.Country)

In [0]:
sharks['Country2'] = [str(x).strip().upper() for x in sharks['Country']]    
sharks['Country'] = sharks['Country2']
sharks = sharks.drop(columns = ['Country2'])

In [0]:
#sharks[sharks['Country'].str.contains(r'.+\?')]
sharks['Country'] = sharks['Country'].str.replace('?', '')

In [0]:
sharks.loc[sharks['Country'] == 'COAST OF AFRICA', 'Country'] = 'AFRICA'
sharks.loc[sharks['Country'] == 'CEYLON (SRI LANKA)', 'Country'] = 'CEYLON'
sharks.loc[sharks['Country'] == 'REUNION', 'Country'] = 'REUNION ISLAND'
sharks.loc[sharks['Country'] == 'MALDIVES', 'Country'] = 'MALDIVE ISLANDS'
sharks.loc[sharks['Country'] == 'UNITED ARAB EMIRATES (UAE)', 'Country'] = 'UNITED ARAB EMIRATES'
sharks.loc[sharks['Country'] == 'ST HELENA, British overseas territory', 'Country'] = 'ST. HELENA'
sharks.loc[sharks['Country'] == 'COLUMBIA', 'Country'] = 'COLOMBIA'
sharks.loc[sharks['Country'] == 'SOUTHWEST PACIFIC OCEAN', 'Country'] = 'SOUTH PACIFIC OCEAN'
sharks.loc[sharks['Country'] == 'FEDERATED STATES OF MICRONESIA', 'Country'] = 'MICRONESIA'
sharks.loc[sharks['Country'] == 'TOBAGO', 'Country'] = 'TRINIDAD & TOBAGO'
sharks.loc[sharks['Country'] == 'WESTERN SAMOA', 'Country'] = 'SAMOA'
sharks.loc[sharks['Country'] == 'MID-PACIFC OCEAN', 'Country'] = 'MID PACIFIC OCEAN' 
sharks.loc[sharks['Country'] == 'SOLOMON ISLANDS', 'Country'] = 'SOLOMON ISLANDS / VANUATU'
sharks.loc[sharks['Country'] == 'VANUATU', 'Country'] = 'SOLOMON ISLANDS / VANUATU'
sharks.loc[sharks['Country'] == 'RED SEA / INDIAN OCEAN', 'Country'] = 'RED SEA'

## Clean 'Year'

In [0]:
sharks = sharks[sharks['Year'].between(1, 1600, inclusive=True) == False]

In [0]:
sharks = sharks[sharks['Date'].str.contains(r'.+B\.C\.') == False]

In [0]:
sharks['Year2'] = sharks['Date'].str.extract(r'([0-9]{4})')
sharks.loc[sharks['Year']==0, 'Year'] = sharks['Year2']

#some of the years extracted above are approximated. ie, 'Before 1905' is too vague to estimate; I've taken 1905 as the year. This could vary in degree of incorrectness- actual may be 1904 or 1704. 
#possibly should be dropped altogether- will decide when viz of year is viewed

sharks.loc[sharks['Year'].isna() == True, 'Year'] = sharks['Year2']
sharks = sharks.drop(columns = ['Year2'])
sharks.loc[sharks['Date'] == 'World War II', 'Year'] = '1940'
sharks['Year'] = sharks['Year'].replace("'", '')
sharks = sharks[sharks['Year'].isna() == False]
sharks['Year'] = sharks['Year'].astype('int')

In [0]:
#sharks['Year'].unique()

## Clean 'Type'

In [0]:
sharks['Type'].unique()

In [0]:
sharks = sharks[sharks['Case Number'] != '2020.04.29']
sharks['Type'][sharks['Type'].isna() == True] = 'Unprovoked'

In [0]:
sharks = sharks[sharks['Type'] != 'Invalid'] #if invalid, will only confuse data
sharks.loc[sharks['Type'] == 'Boat', 'Type'] = 'Watercraft' # will actually switch to 'Watercraft' for consistency
sharks = sharks[sharks['Type'] != 'Under investigation'] # there is only one so just going to drop it
sharks = sharks[sharks['Type'] != 'Unconfirmed'] # there is only one so just going to drop it- missing much info so don't want to coerce into unprovoked
sharks.loc[(sharks['Type'].isna() == True) & (sharks['Activity'] == 'Wreck of a sampam'), 'Type'] = 'Sea Disaster'
sharks.loc[sharks['Type'] == 'Unverified', 'Type'] = 'Unprovoked' #only 1; acting as if legit

## Clean 'Activity'

In [0]:
set(sharks['Activity'])

In [0]:
#set(sharks['Activity'][(sharks['Activity'].str.contains(r'(swim|stand|wading|Swim|Stand|Wading)')) & (~sharks['Activity'].str.contains(r'(Fish|fish|apsize|shark|Shark|porpoise|oyster|whale|dolphin|perlemoen|abalone|slaughterhouse)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False))])
sharks['Activity_clean'] = ''
# includes: (Attempt|Attack|harpooned|dead|Catching|Chumming|captured|force-feed|Kiss|feeding|photographing|Dragging|filming|Feeding|Feeling|Filming|Finning|Fell|Fishing|Gaffing|Grab|Hand feed|Harass|Harpoon|Hauling|Help|Hoist|Hold|Hunt|Inspect|Investigat|landed on|Killing|fishing|Measuring|Moving|Netting|Observing|Petting|Photographing|Picking|Pulling|foot|hand|Removing|Restrain|Reviv|catch|Shooting|Grab|Slap|lasso|pulled|shot|Spearing|Standing|stepped|Stuffing|feeding|Swimming with|Tagging|Teasing|Testing|Thrashing|Touching|netted|Watching|Wrangling|)

sharks.loc[(sharks['Activity'].str.contains(r'(collided|[C|c]apsize|aground|[S|s]hipwreck|210-ton brig|[W|w]reck|[D|d]isaster|[A|a]ccident|founder|crash|overturn|stove|sank)', na=False)) | (sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_clean'] = 'Sea disaster/capsize/wreck/accident'
sharks.loc[(sharks['Activity'].str.contains(r'( fish|[F|f]ish|[C|c]ollecting|[C|c]lam|porpoise|whale|dolphin|perlemoen|[A|a]balone|crayfish|slaughterhouse|bait|[T|t]rochus|opihi|squid|shad|bluefish|net|[C|c]rab|conch|beche-de-mer|trepang|tuna|shellfish|[L|l]obster|[S|s]hrimp|[P|p]rawn|meat|turtle|seal|[O|o]yster|[A|a]ngling|sardine|bichiques|[C|c]hum|[S|s]almon|[F|f]lounder|sea urchins|sponges)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)) & (~sharks['Activity'].str.contains(r'Hawaiian brig|Observing', na=False)), 'Activity_clean'] = 'Fishing / Proximity to Sea Life'
sharks.loc[(sharks['Activity'].str.contains(r'(SUP|[Ss]wim|[Ss]ta[mn]d|[Ww]ad[e|ing]|[Bb]athing|[D|d]iv[e|ing]|[S|s]urf|[W|w]alk|[S|s]it|[B|b]oard|Playing|Snorkeling|Floating|[F|f]eet|Cling|Clean|Lying|Adrift|Kneeling|Holding|Paddling|Splash|Wash|[F|f]ell|Crouch|banana|Filming$|documentary|Jump|Rest|Sitting|Tread|[S|s]ki)')) & (~sharks['Activity'].str.contains(r'(Chasing|[C|c]apsize|aground|[S|s]hipwreck|210-ton brig|[W|w]reck|[D|d]isaster|[A|a]ccident|feeding| fish|[F|f]ish|[C|c]lam|porpoise|oyster|whale|dolphin|perlemoen|[A|a]balone|crayfish|slaughterhouse|bait|trochus|opihi|squid|bluefish|[C|c]rab|conch|beche-de-mer|trepang|tuna|shellfish|[L|l]obster|[S|s]hrimp|meat|turtle|seal|[O|o]yster|[C|c]hum|[S|s]almon|sea urchins|sponges|Feeding|holding shark|putting hand|shark tank|stepped on|grabbed shark)', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_clean'] = 'Unprovoked' #'Swimming/Wading/Standing/Diving etc'
sharks.loc[(sharks['Activity'].str.contains(r'(shark|Shark)', na=False)) & (~sharks['Activity'].str.contains(r'surfmat|bluefish|squid|Fukulya Maru|bathing|capsize|Crossing inlet|trochus|wife|tarpon|dory|Len Bedford|grouper|opihi|menaced|warn bathers|possibly ascended into path', na=False)) & (~sharks['Type'].str.contains('Sea Disaster', na=False)), 'Activity_clean'] = 'Provoked'
sharks.loc[sharks['Activity'] == 'Chasing shark out of bathing area while riding on a surf-ski', 'Activity_clean'] = 'Provoked'

sharks.loc[sharks['Activity'].str.contains(r'([Mm]urder|[Ss]uicide)', na=False), 'Activity_clean'] = 'Invalid'

sharks.loc[(sharks['Activity'].str.contains('[C|c]anoeing', na=False)) | (sharks['Activity'] == 'Kayaking') | (sharks['Activity'] == 'Kakaying') | (sharks['Activity'] == 'Kayaking ') | (sharks['Activity'] == 'Boat') | (sharks['Activity'] == 'Rowing') | (sharks['Activity'] == 'Sculling') | (sharks['Activity'] == 'Sailing') | (sharks['Activity'] == 'Watercraft') | (sharks['Activity'] == 'Transatlantic Rowing') | (sharks['Activity'] == 'Cruising') | (sharks['Activity'] == 'Rowing') | (sharks['Activity'] == 'Yacht race') | (sharks['Activity'].str.contains('Rowing', na=False)) | (sharks['Name'].str.contains('dinghy|[B|b]oat|ferry|yacht|skiff|cutter')) | (sharks['Activity'].str.contains('dinghy|[B|b]oat|ferry|yacht|skiff|cutter')), 'Activity_clean'] = 'Watercraft'

In [0]:
set(sharks['Activity'][sharks['Activity_clean'] == ''])
#sharks.groupby(['Activity', 'Activity_collapsed']).size()

In [0]:
sharks['Activity_clean'].unique()
#set(sharks['Activity'][sharks['Activity_clean'] == 'Fishing / Proximity to Sea Life'])
#set(sharks['Activity'][sharks['Activity_clean'] == 'Provoked'])
#set(sharks['Activity'][sharks['Activity_clean'] == 'Sea disaster/capsize/wreck/accident'])
#set(sharks['Activity'][sharks['Activity_clean'] == 'Unprovoked']) 
#set(sharks['Activity'][sharks['Activity_clean'] == 'Watercraft'])

In [0]:
#sharks[['Activity', 'Type', 'Name']][(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == False)] 

#OKAY I am tired of cleaning this data.... went through results of above ^ and final assignations are fine. Gonna trust the few that seem either way from Activity since I am not going through attack pdf info
sharks.loc[(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == False), 'Activity_clean'] = sharks.loc[(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == False), 'Type']

In [0]:
#fill nans
#going to have to take their work for it.....tempted to drop due to lack of info, but it's not a huge dataset. Then again, that means if they are wrong they will have a larger impact
len(sharks[['Activity','Type']][(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == True)])

sharks.loc[(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == True), 'Activity_clean'] = sharks.loc[(sharks['Activity_clean'] == '') & (sharks['Activity'].isna() == True), 'Type'] = sharks['Type'].astype(str) + '_NODETAIL'

In [0]:
# lets make sure incidents where ppl grabbed sharks are labelled provoked....
sharks.loc[sharks['Injury'].str.lower().str.contains(r'(stepped.*|spear.*|dead|netted|shot.*|grabbed|frighten|snared|chasing|snared|kicked|land[ed|ing]|seized|removing|hooked|hit|stepping on|captive|dying|captured|gaffed|boated|fell on|trod on the|secure|roping|poked|harpooned|struck|noosed)(?= shark)', na=False), 'Activity_clean' ] = 'Provoked'

sharks.loc[sharks['Injury'].str.lower().str.contains(r'(?<=shark).*(catch|grabbed|capture|trapped|shot|drag|patted|caught|gaff|taken onboard|brought onboard|holding)', na=False), 'Activity_clean'] = 'Provoked'

#sharks['Injury'].loc[sharks['Injury'].str.lower().str.contains('provoked', na=False), 'Activity_clean'] = 'Provoked' 
# do i trust their definition and categorization of provoked???

In [0]:
#Will explore data further then decide whether to keep.
#sharks[['Type', 'Activity']][sharks['Activity_clean'] == 'Provoked']
#sharks[['Type', 'Activity']][sharks['Activity_clean'] == 'Questionable']
#sharks[['Type', 'Activity']][sharks['Activity_clean'] == 'Unprovoked']
#sharks[['Type', 'Activity']][sharks['Activity_clean'] == 'Watercraft']

#set(sharks['Activity_clean'])

## Clean 'Sex'

In [0]:
#set(sharks['Area'])
#set(sharks['Location'])
# set(sharks['Sex '])

In [0]:
# ORIGINAL

sharks.loc[sharks['Sex '] == 'F', 'Sex_clean'] = 'F'
sharks.loc[sharks['Sex '] == 'M', 'Sex_clean'] = 'M'

# FEMALE
sharks.loc[sharks['Name'].str.contains('female', na=False), 'Sex_clean'] = 'F'

# MALE
sharks.loc[(sharks['Sex '] == 'M ') | (sharks['Sex '] == 'lli') | (sharks['Name'].str.contains('(?<![F|f]e)male|schoolboy')), 'Sex_clean'] = 'M'
sharks.loc[sharks['Name'].str.contains(r'^Frederic|^Jose|^Laurent|^Dr. Leo|^Sergio|^Dr. George|^Ian|^Tony|^Vittorio|^Captain Jack|^Maurice|^William|^John|^Emile|^Jeff|^Miller|^male|^Gilvan|^Shawn|^Stephen|^Yuji|^Josebias|^Ben|^Jurandir|^Nagisa|^Yuji', na=False), 'Sex_clean'] = 'M'

# UNKNOWN
sharks.loc[(sharks['Name'].isna()) & (sharks['Sex '].isna()), 'Sex_clean'] = 'U' 
sharks.loc[(sharks['Name'].str.contains(r'^[A-Z]\.')) | (sharks['Sex '] == '\.') | (sharks['Name'].str.contains('child|teen|Unknown|Unidentified|Anonymous|student|teacher|aquarist|swimmer|anonymous|a native|a resident|a surfer|U.S. citizen|Fimler|^a youth', na=False)), 'Sex_clean'] = 'U'

# MULTI
sharks.loc[(sharks['Name'].str.contains(r'[O|o]ccupants|[P|p]assengers|[M|m]igrants|[R|r]efugees|[P|p]oachers|[P|p]eople|[R|r]owers|[C|c]rew|pilgims|Fijians|slaves|youths|passenger|tourists|fishermen|friends|swimmers|men|pilgrims|bathers|\&|Swimming', na=False)) | (sharks['Sex '] == 'M x 2'), 'Sex_clean'] = 'Multi'
sharks.loc[sharks['Case Number'].str.contains('1934.07.11|1959.07.03.a & b', na=False), 'Sex_clean'] = 'Multi' # M&F

#SHIPS/NA
sharks.loc[(sharks['Activity_clean'] == 'Watercraft') | (sharks['Name'].str.contains(r'dinghy|[B|b]oat|ferry|yacht|skiff|cutter|paddle|launch|Hubeh|Leonida|trawler') & (~sharks['Name'].str.contains(r'[C|c]rew', na=False))), 'Sex_clean'] = 'N/A'

In [0]:
print(len(sharks[sharks['Sex_clean'].isna() == True]))
set(sharks['Sex_clean'])

## Clean 'Species'

In [0]:
sharks['Species_clean'] = ['' for x in np.arange(len(sharks['Species ']))]
sharks = sharks.reset_index()
sharks['Species '] = sharks['Species '].fillna('NA')

In [0]:
for x in np.arange(len(sharks)):
    if re.search('[S|s]even[\s|\-]?gill|7[\s|\-]?gill' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] = 'Seven-gill shark'
    elif re.search('[T|t]hresher' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] = 'Thresher shark'
    elif re.search('[W|w]obbegong' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] = 'Wobbegong shark'
    elif re.search('[L|l]emon' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] =  'Lemon shark'
    elif re.search('[L|l]eopard' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] =  'Leopard shark'
    elif re.search('[H|h]ammerhead' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] =  'Hammerhead shark'
    elif re.search('(?<![S|s]and )[T|t]iger' , sharks.loc[x, 'Species ']):
        sharks.loc[x,  'Species_clean'] = 'Tiger shark'
    elif re.search('(?<!([G|g]r[e|a]y|[B|b]lue)[\s|\-])[N|n]urse' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Nurse shark'
    elif re.search('[B|b]lack[\s|-]?tip' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Blacktip shark'
    elif re.search('[S|s]pinner' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Spinner shark'
    elif re.search('[D|d]usky' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Dusky shark'
    elif re.search('[B|b]ull|[Z|z]ambe[z|s]i|C\. leucas|Lake Nicaragua' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Bull shark'
    elif re.search('[B|b]ronze|(?<![W|w]hite[\s|\-]tipped\s])[W|w]haler|[C|c]opper|[N|n]arrow[\s|\-]?tooth' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Copper shark'
    elif re.search('[B|b]lack[\s|-]?fin' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Blackfin shark'
    elif re.search('[B|b]lue[\s|-]?nose|blunt[\s|\-]?nose|six[\s|\-]?gill|6[\s|\-]?gill' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] =  'Bluntnose sixgill shark'
    elif re.search('(?<!([[L|l]esser\s))[W|w]hite |[W|w]hite pointer' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Great white shark'
    elif re.search('[M|m]ako|[B|b]lue pointer|[B|b]onit[a|o]' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Mako shark'
    elif re.search('[P|p]orbeagle' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Porbeagle shark'
    elif re.search('(?<=([G|g]r[e|a]y|[B|b]lue)[\s|\-])[N|n]urse|[W|w]hite[\s|\-]?tip|[N|n]igano|[S|s]ilver[\s|\-]?tip|brown|(?<![B|b]ronze\s)[W|w]haler|(?<![S|s]potted )[R|r]agged[\s|\-]?tooth|[S|s]and[\s]?bar' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Oceanic whitetip shark'
    elif re.search('[D|d]og|mud|piked|dogfish' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Spiny dogfish'
    elif re.search('(?<![G|g]r[e|a]y\s)[R|r]eef' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Caribbean reef shark'
    elif re.search('[G|g]r[e|a]y\s(?!nurse)(?!whaler)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Grey reef shark'
    elif re.search('C\.\salbimarginatus|[S|s]ilver[\s|\-]?tip' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Silvertip shark'
    elif re.search('[S|s]ilk|blackspot|[G|g]r[a|e]y\s(?=whaler)|olive|[R|r]idgeback|[S|s]ickle' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Silky shark'
    elif re.search('[G|g]ummy' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Gummy shark'
    elif re.search('[C|c]ow' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Cow shark'
    elif re.search('[C|c]arpet' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Carpet shark'
    elif re.search('[B|b]lue(?![\-|\s]{0,1}tip)(?!\s{1}pointer)(?![\s|\-]{1}nurse)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Blue shark'
    elif re.search('(?<=[B|b]lue )[N|n]urse|(?<=[G|g]r[e|a]y )[N|n]urse|[S|s]and(?!bar)|(?<=[S|s]potted )[R|r]agged[\s|\-]?tooth' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Sand shark'
    elif re.search('[B|b]anjo' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Banjo shark - not a real shark'
    elif re.search('[S|s]hovelnose' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Shovelnose shark - not a real shark'
    elif re.search('[S|s]mooth[\s|\-]?hound' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Smooth-hound shark'
    elif re.search('[G|g]oblin' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Goblin shark'
    elif re.search('[C|c]ookie[\s|\-]?cutter|[C|c]igar' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Cookiecutter shark'
    elif re.search('[G|g]alapagos' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Galapagos shark'
    elif re.search('[C|c]atshark' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Catshark'
    elif re.search('[P|p]ort [J|j]ackson' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Port Jackson shark'
    elif re.search('[A|a]ngel|[M|m]onkfish' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Angelshark'
    elif re.search('[W|w]hale(?<!er)' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'Whale shark'
    elif re.search('[S|s]oupfin|[S|s]chool|[S|s]napper|[T|t]ope' , sharks.loc[x, 'Species ']): 
        sharks.loc[x,  'Species_clean'] = 'School shark'
#    elif re.search('' , sharks.loc[x, 'Species ']): 
#        sharks.loc[x,  'Species'] = ''
    elif sharks.loc[x, 'Species '] == 'NA':
         sharks.loc[x,  'Species_clean'] = 'No details'
    else:
        ''

In [0]:
# none are missing :) although some are overlapping (I know some had multiple types of sharks in the string- something to be looked at)
sharks.loc[sharks['Species_clean'] == '', 'Species_clean'] = 'No details'

set(sharks['Species_clean'])

In [0]:
# Note to my Canadian-metric-self:

# ' = feet 
# " = inches

In [0]:
sharks['Size_min'] =  [np.nan for x in np.arange(len(sharks['Species ']))]
sharks['Size_max'] =  [np.nan for x in np.arange(len(sharks['Species ']))]

In [0]:
sharks[['Species ','Size_min','Size_max']].head()

In [0]:
for x in np.arange(len(sharks)):
    #no metres, no inches, has feet
    if re.search('[0-9]+\.?[0-9]?\s?m' , sharks.loc[x, 'Species ']) is None and re.search('[0-9]\.?[0-9]?\'{2}|\"{1}' , sharks.loc[x, 'Species ']) is None and bool(re.search('[0-9]\.?[0-9]?\'{1}(?!\')' , sharks.loc[x, 'Species '])):
        if len(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_max'] = float(re.search('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species ']).group(1)) * 0.3028
        elif len(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])) == 2:
            sharks.loc[x,  'Size_min'] = float(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])[0]) * 0.3028
            sharks.loc[x,  'Size_max'] = float(re.findall('([0-9]\.?[0-9]?)\'{1}(?!\')' , sharks.loc[x, 'Species '])[1]) * 0.3028
        else:
            sharks.loc[x,  'Size_max'] = np.nan
    #has metres
    elif bool(re.search('[0-9]+\.?[0-9]?\s?m' , sharks.loc[x, 'Species '])) and (re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])) is None:
        if len(re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_max'] = re.search('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species ']).group(1)
        elif len(re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])) == 2:
            sharks.loc[x,  'Size_min'] = re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])[0]
            sharks.loc[x,  'Size_max'] = re.findall('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species '])[1]
        else:
             sharks.loc[x,  'Size_max'] = np.nan
        #first match having no m suffix:
    elif len(re.findall('[0-9]+\.?[0-9]*\s?m' , sharks.loc[x, 'Species '])) == 1 &  len(re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])) == 1:
            sharks.loc[x,  'Size_min'] = re.findall('([0-9]+\.?[0-9]?)\s?(?=[\-|to])' , sharks.loc[x, 'Species '])[0]
            sharks.loc[x,  'Size_max'] = re.search('([0-9]+\.?[0-9]?)\s?m' , sharks.loc[x, 'Species ']).group(1)
    else:
        sharks.loc[x,  'Size_max'] = np.nan

In [0]:
sharks.loc[sharks['Species '] == 'White shark 25 to 3 m', 'Size_max'] = 30

## Clean 'Injury'

In [0]:
#set(sharks['Injury'].to_list())

In [0]:
sharks['Injury_clean'] =  [np.nan for x in np.arange(len(sharks['Species ']))]
sharks.loc[sharks['Injury'].str.contains('no injury', na=False), 'Injury_clean'] = 'No injury'
sharks.loc[sharks['Injury'].str.lower().str.contains('fatal', na=False), 'Injury_clean'] = 'Fatal'
sharks['Injury_clean'] = sharks['Injury_clean'].fillna('Injury')

#maybe worth using tokenization to identify limbs vs torso injuries at some point (but i dont want to right now, lets keep moving!)

> ## Clean 'Date'

In [0]:
sharks['Date_clean'] =  [np.nan for x in np.arange(len(sharks['Species ']))]
sharks['Year_clean'] =  [np.nan for x in np.arange(len(sharks['Species ']))]

In [0]:
sharks = sharks.loc[sharks['Date'] != 'Jun-1018']
sharks = sharks.loc[~sharks['Date'].str.contains(r'Before|Fall|Summer|Early [0-9]{4}')]
sharks = sharks.reset_index(drop=True)
sharks['Date'] = [sharks.loc[x,'Date'].replace('Reported', '') for x in np.arange(len(sharks['Date']))]

In [0]:
for x in np.arange(len(sharks)):
    if isinstance(sharks.loc[x, 'Date_clean'], datetime.date):
        pass
    elif bool(re.search(r'[0-9]{2}[- .]{1}[A-z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])):
        sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{2}.{1}[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0].replace('.', '-')
    elif bool(re.search('^[A-z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])):
        sharks.loc[x, 'Date_clean'] = '15-' + re.findall('^[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('(?<![0-9])[0-9]{1}[- .]{1}[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])):
        sharks.loc[x, 'Date_clean'] = '0' + re.findall('(?<![0-9])[0-9]{1}.{1}[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('[0-9]{2}[- .]{1}[a-zA-Z]{3}[0-9]{4}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{2}.{1}[a-zA-Z]{3}', sharks.loc[x, 'Date'])[0]  + '-' + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('[M|m]id.{1}[a-zA-Z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = '15-' + re.findall('[a-zA-Z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif sharks.loc[x, 'Date'].strip() == '26-Sep-t937':
         sharks.loc[x, 'Date_clean'] = '26-Sep-1937'
    elif sharks.loc[x, 'Date'] == '22-Jul-144':
         sharks.loc[x, 'Date_clean'] = '22-Jul-1944'
    elif sharks.loc[x, 'Date'] == '02-Ap-2001':
         sharks.loc[x, 'Date_clean'] = '02-Apr-2001'
    elif sharks.loc[x, 'Date'] == '30-March-1878':
         sharks.loc[x, 'Date_clean'] = '30-Mar-1878'
    elif bool(re.search('July', sharks.loc[x, 'Date'])):
        if bool(re.findall('[0-9]{2}[- .]{1}[A-z]{3}', sharks.loc[x, 'Date'])):
            sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{2}.{1}[A-z]{3}', sharks.loc[x, 'Date'])[0] + '-' + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0]
        else:
            sharks.loc[x, 'Date_clean'] = '15-Jul-' + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('December', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = '15-Dec-' + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('Early', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = '01-' + re.findall('[A-z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('Late', sharks.loc[x, 'Date'])):
        if bool(re.search('Late [A-z]{3}', sharks.loc[x, 'Date'])):
                sharks.loc[x, 'Date_clean'] = '30-' + re.findall('[A-z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
        else:   
            sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0][:3] + '9'       
    elif bool(re.search('[0-9]{4}.8{1}[0-9]{2}.{1}[0-9]{2}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = [re.findall('(?<![0-9])[0-9]{2}(?!.)', sharks.loc[x, 'Date'])[0] if len(re.findall('(?<![0-9])[0-9]{2}(?!.)', sharks.loc[x, 'Date'])) >=1 else '15'] + [re.search('\.{1}[0-9]{2}\.{1}', sharks.loc[x, 'Date']) if re.search('\.{1}[0-9]{2}\.{1}', sharks.loc[x, 'Date']) == '.00.' else '.06.'] + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0].split()
    elif bool(re.search('[0-9]{2}.{1}[A-z]{3}.{2}[0-9]{4}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{2}.{1}[A-z]{3}', sharks.loc[x, 'Date'])[0] + re.findall('.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('[0-9]{2}.{2}[A-z]{3}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{2}.{1}', sharks.loc[x, 'Date'])[0] + re.findall('[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('[A-z]{3}[- .]{1}[0-9]{2}[- .]{1}[0-9]{4}', sharks.loc[x, 'Date'])):
         sharks.loc[x, 'Date_clean'] = re.findall('(?<![0-9])[0-9]{2}.{1}', sharks.loc[x, 'Date'])[0] + re.findall('[A-z]{3}.{1}', sharks.loc[x, 'Date'])[0] + re.findall('[0-9]{4}', sharks.loc[x, 'Date'])[0]
    elif bool(re.search('[0-9]{3}[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])):
        sharks.loc[x, 'Date_clean'] = re.findall('[0-9]{3}[A-z]{3}', sharks.loc[x, 'Date'])[0][:2] + '-' + re.findall('[A-z]{3}.{1}[0-9]{4}', sharks.loc[x, 'Date'])[0].replace('.', '-')
    else:
        sharks.loc[x, 'Date_clean'] = ''

In [0]:
for x in np.arange(len(sharks)):
    if len(re.findall('[0-9]{4}', sharks.loc[x, 'Date_clean'])) == 1:
        sharks.loc[x, 'Year_clean'] = re.search('[0-9]{4}', sharks.loc[x, 'Date_clean']).group(0)
    elif (len(re.findall('[0-9]{4}', sharks.loc[x, 'Date_clean'])) == '') & (len(re.findall('[0-9]{4}', sharks.loc[x, 'Year'].astype('str'))) == 1):
        sharks.loc[x, 'Year_clean'] = re.search('[0-9]{4}', sharks.loc[x, 'Year']).group(0)
    elif len(re.findall('[0-9]{4}', sharks.loc[x, 'Date'])) >= 1:
        sharks.loc[x, 'Year_clean'] = re.search('[0-9]{4}', sharks.loc[x, 'Date']).group(0)
    elif len(re.findall('[mid-]?[0-9]{4}s', sharks.loc[x, 'Date'])) >= 1 & (sharks.loc[x, 'Date_clean'] == ''):
        sharks.loc[x, 'Year_clean'] = re.search('[0-9]{4}', sharks.loc[x, 'Date']).group(0)[:3] + '5'
    elif len(re.findall('Wo[i]?rld War II', sharks.loc[x, 'Date'])) == 1 & (sharks.loc[x, 'Date_clean'] == ''):
        sharks.loc[x, 'Year_clean'] = '1940'
    else:
        sharks.loc[x, 'Year_clean'] = np.nan

In [0]:
# check occasions where original year and cleaned year do not match for validation

sharks.loc[sharks['Date_clean'] == '', 'Date_clean'] = np.nan


print(sharks[['Year_clean']][(sharks['Year'].astype(str)) != (sharks['Year_clean'].astype(str))].count())
sharks[['Date', 'Date_clean', 'Year', 'Year_clean']][(sharks['Year'].astype(str)) != (sharks['Year_clean'].astype(str))]

In [0]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i in sharks['Date_clean'].astype(str):
    if re.search('[A-z]{3}', i)[0] in months:
        pass
    elif re.search('[A-z]{3}', i)[0] == 'nan':
        pass
    else:
        print(i)

In [0]:
sharks['Date'][sharks['Date'].str.contains('ate') == True]

In [0]:
sharks['Month'] = pd.DatetimeIndex(sharks['Date_clean']).month
sharks['Day'] = pd.DatetimeIndex(sharks['Date_clean']).day

## Clean 'Time'

In [0]:
sharks['Time_clean'] =  [np.nan for x in np.arange(len(sharks['Time']))]

In [0]:
for x in np.arange(len(sharks)):
    if len(re.findall('[0-9]{2}h[0-9]{2}', str(sharks.loc[x, 'Time']))) != 0:
        sharks.loc[x, 'Time_clean'] = re.sub('h', ':', re.search('[0-9]{2}h[0-9]{2}', sharks.loc[x, 'Time']).group(0))
    else:
        sharks.loc[x, 'Time_clean'] = np.nan

In [0]:
sharks.to_excel('sharks_clean.xlsx', index=False)